In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pmdarima import auto_arima
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_percentage_error
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
import math
from statsmodels.tsa.seasonal import seasonal_decompose


In [20]:
final_data = pd.read_csv('./final_data.csv')
result2025 = pd.read_csv('./result_sarima_arima.csv')


In [21]:
historical_data = final_data
forecast_data = result2025

In [24]:
import pandas as pd

# Ініціалізація оновленого датасету
updated_data = historical_data.copy()

# Обробка прогнозів для кожної комбінації країни, вікової групи і статі
for _, group in forecast_data.groupby(["Country", "Age", "Sex"]):
    # Вибір найкращої моделі для цієї комбінації
    best_model = group.loc[group['RMSE'].idxmin()]

    # Фільтрація даних для найкращої моделі
    best_forecast = group[group['Model'] == best_model['Model']]

    # Перетворення дат
    best_forecast = best_forecast.rename(columns={"Date": "Year-Month"})
    best_forecast['Year'] = pd.to_datetime(best_forecast['Year-Month']).dt.year
    best_forecast['Month'] = pd.to_datetime(best_forecast['Year-Month']).dt.month

    # Вибір потрібних колонок
    best_forecast = best_forecast[["Age", "Sex", "Country", "Forecast", "Year", "Month"]]
    best_forecast = best_forecast.rename(columns={"Forecast": "Unemployment Rate (%)"})

    # Додавання прогнозів до оновленого датасету
    updated_data = pd.concat([updated_data, best_forecast], ignore_index=True)

# Збереження оновленого датасету
updated_data.to_csv("updated_data.csv", index=False)

# Перевірка результату
print(updated_data.tail())


       Age    Sex                   Country  Unemployment Rate (%)  Year  \
32367  25+  Total  United States of America               0.029308  2025   
32368  25+  Total  United States of America               0.029101  2025   
32369  25+  Total  United States of America               0.028894  2025   
32370  25+  Total  United States of America               0.028688  2025   
32371  25+  Total  United States of America               0.028481  2025   

       Month  
32367      8  
32368      9  
32369     10  
32370     11  
32371     12  


In [11]:
initial_data = final_data
model_performance = result2025

In [12]:


# Display the first few rows of the datasets for verification
print("Initial Data Sample:")
print(initial_data.head())
print("\nModel Performance Data Sample:")
print(model_performance.head())

# -----------------------------
# Step 2: Select the Best Model
# -----------------------------

# Ensure that 'RMSE' column exists
if 'RMSE' not in model_performance.columns:
    raise KeyError("The model performance dataset must contain an 'RMSE' column.")

# Select the best model per group based on the lowest RMSE
# Assuming that each row in model_performance corresponds to a forecast from a specific model

# Group by Country, Age, Sex and select the model with the minimum RMSE within each group
best_models = model_performance.loc[model_performance.groupby(['Country', 'Age', 'Sex'])['RMSE'].idxmin()]

# Display the selected best models
print("\nBest Models per Group:")
print(best_models[['Country', 'Age', 'Sex', 'Model', 'RMSE']])

# -----------------------------
# Step 3: Prepare Forecast Data
# -----------------------------

# We'll assume that 'result_sarima_arima.csv' contains both model performance metrics and forecasted values
# If forecasts are in a separate file, adjust the code accordingly

# Extract forecasted data from the best models
# Assuming that forecasted values are in the 'Forecast' column and correspond to the 'Date'

# Create a DataFrame to hold all forecasted data
forecast_data = best_models[['Country', 'Age', 'Sex', 'Date', 'Forecast']].copy()

# Convert 'Date' to datetime
forecast_data['Date'] = pd.to_datetime(forecast_data['Date'])

# Extract Year and Month from Date
forecast_data['Year'] = forecast_data['Date'].dt.year
forecast_data['Month'] = forecast_data['Date'].dt.month

# Rename 'Forecast' to 'Unemployment Rate (%)' to match the initial dataset
forecast_data.rename(columns={'Forecast': 'Unemployment Rate (%)'}, inplace=True)

# Select and reorder columns to match the initial dataset
forecast_data = forecast_data[['Age', 'Sex', 'Country', 'Unemployment Rate (%)', 'Year', 'Month']]

# Display the forecast data sample
print("\nForecast Data Sample:")
print(forecast_data.head())

# -----------------------------
# Step 4: Append Forecasts to Initial Data
# -----------------------------

# Ensure that both datasets have the same data types for merging
# For example, ensure 'Age', 'Sex', 'Country' are of type string
for col in ['Age', 'Sex', 'Country']:
    initial_data[col] = initial_data[col].astype(str)
    forecast_data[col] = forecast_data[col].astype(str)

# Similarly, ensure 'Year' and 'Month' are integers
initial_data['Year'] = initial_data['Year'].astype(int)
initial_data['Month'] = initial_data['Month'].astype(int)
forecast_data['Year'] = forecast_data['Year'].astype(int)
forecast_data['Month'] = forecast_data['Month'].astype(int)

# Concatenate the initial data with the forecast data
combined_data = pd.concat([initial_data, forecast_data], ignore_index=True)

# Sort the combined data for better readability
combined_data.sort_values(by=['Country', 'Age', 'Sex', 'Year', 'Month'], inplace=True)

# Reset the index
combined_data.reset_index(drop=True, inplace=True)

# Display the combined data sample
print("\nCombined Data Sample:")
print(combined_data.tail())

# -----------------------------
# Step 5: Save the Combined Dataset
# -----------------------------
#
# # Save the combined dataset to a new CSV file
# combined_data.to_csv(output_data_path, index=False)
#
# print(f"\nForecasts have been successfully integrated into the initial dataset and saved to '{output_data_path}'.")


Initial Data Sample:
   Age     Sex  Country  Unemployment Rate (%)  Year  Month
0  25+  Female  Ireland                  0.096  2014      4
1  25+   Total   Norway                  0.028  2014     10
2  25+    Male  Iceland                  0.052  2021      1
3  25+    Male  Iceland                  0.049  2021      2
4  25+    Male  Iceland                  0.048  2021      3

Model Performance Data Sample:
    Model Country    Age   Sex        Date  Forecast       R^2      MSE  \
0  SARIMA   Italy  15-24  Male  2025-01-01  0.181034  0.979012  0.00006   
1  SARIMA   Italy  15-24  Male  2025-02-01  0.181034  0.979012  0.00006   
2  SARIMA   Italy  15-24  Male  2025-03-01  0.181034  0.979012  0.00006   
3  SARIMA   Italy  15-24  Male  2025-04-01  0.171053  0.979012  0.00006   
4  SARIMA   Italy  15-24  Male  2025-05-01  0.171053  0.979012  0.00006   

      RMSE    MAPE  
0  0.00777  0.0114  
1  0.00777  0.0114  
2  0.00777  0.0114  
3  0.00777  0.0114  
4  0.00777  0.0114  

Best Mode

In [13]:
combined_data


,Age,Sex,Country,Unemployment Rate (%),Year,Month
0,15-24,Female,Algeria,0.366337,2014,3
1,15-24,Female,Algeria,0.413681,2014,9
2,15-24,Female,Algeria,0.453020,2015,3
3,15-24,Female,Algeria,0.453020,2015,9
4,15-24,Female,Algeria,0.399240,2016,3
...,...,...,...,...,...,...
30923,25+,Total,Uruguay,0.058608,2024,2
30924,25+,Total,Uruguay,0.062042,2024,3
30925,25+,Total,Uruguay,0.059679,2024,4
30926,25+,Total,Uruguay,0.058946,2024,5


In [14]:
combined_data.to_csv('./result_with_2025.csv', index=False)
